In [22]:
import spotlight
annotations = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate',
    'Berlin is the capital and largest city of Germany',confidence=0.4,support = 20)
annotations

[{'URI': 'http://dbpedia.org/resource/Berlin',
  'support': 87107,
  'types': 'Wikidata:Q515,Wikidata:Q486972,Schema:Place,Schema:City,DBpedia:Settlement,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:City',
  'surfaceForm': 'Berlin',
  'offset': 0,
  'similarityScore': 0.9983656061097194,
  'percentageOfSecondRank': 0.0008228013211064525},
 {'URI': 'http://dbpedia.org/resource/City',
  'support': 22610,
  'types': '',
  'surfaceForm': 'city',
  'offset': 34,
  'similarityScore': 0.8720405806057635,
  'percentageOfSecondRank': 0.10547609042001196},
 {'URI': 'http://dbpedia.org/resource/Germany',
  'support': 222752,
  'types': 'Wikidata:Q6256,Schema:Place,Schema:Country,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:Country',
  'surfaceForm': 'Germany',
  'offset': 42,
  'similarityScore': 0.9959557085639706,
  'percentageOfSecondRank': 0.0024005027968948783}]

In [23]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
endpoint = "https://fr.dbpedia.org/sparql"
def sparql_to_dataframe(endpoint, query):
    """
    Convert SPARQL results into a Pandas DataFrame.
    Credit: https://lawlesst.github.io/notebook/sparql-dataframe.html
    """
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()

    cols = results['head']['vars']
    out = []
    for row in results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)


In [30]:
def test():
    return '''
select * where {
    <http://dbpedia.org/resource/Germany> rdfs:comment ?y.
 }'''

In [26]:
def retrieveTest(endpoint) :
    query = test()
    result = sparql_to_dataframe(endpoint, query)
    return result

In [31]:
retrieveTest(endpoint)

,y
